In [28]:
import sys

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

sys.path.append(r'C:\Users\19167\AppData\Local\Programs\Python\smartone_quant')

In [ ]:
# 获取股票行情数据
"""
"time"                #时间戳
"open"                #开盘价
"high"                #最高价
"low"                 #最低价
"close"               #收盘价
"volume"              #成交量
"amount"              #成交额
"settle"              #今结算
"openInterest"        #持仓量
"""

In [29]:
from typing import List
import pandas as pd

# 日期工具
from datetime import datetime, timedelta
from xtquant import xtdata

from app.models.qmt_stock_daily import QmtStockDailyOri

In [ ]:
# 测试下载000001.SZ的日 k 行情数据并获取打印
stock_code: str = '000001.SZ'
print(f'开始下载股票{stock_code}的行情数据')
# 获取一年前的日期,yyyyMMdd格式
one_years_ago = (datetime.now() - timedelta(days=1 * 365)).strftime('%Y%m%d')
print(f'一年前的日期为：{one_years_ago}')
# 获取两年前的日期,yyyyMMdd格式
two_years_ago = (datetime.now() - timedelta(days=2 * 365)).strftime('%Y%m%d')
print(f'两年前的日期为：{two_years_ago}')
# 获取三年前的日期,yyyyMMdd格式
three_years_ago = (datetime.now() - timedelta(days=3 * 365)).strftime('%Y%m%d')
print(f'三年前的日期为：{three_years_ago}')
# 获取今天日期
today = datetime.now().strftime('%Y%m%d')
print(f'今天的日期为：{today}')

In [ ]:
"""
get_market_data函数返回数据格式为：
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""


# 公共解析数据的方法，将stock_data解析为股票对象
def parse_stock_data(stock_data: dict, model_cls = QmtStockDailyOri) -> list:
    """
    解析股票行情数据，支持日/周/月K模型
    :param stock_data: 股票行情数据
    :param model_cls: 股票数据模型类
    :return: 股票对象列表
    """
    stock_list = []
    stock_data_time = stock_data['time']
    stock_codes = stock_data_time.index.tolist()
    stock_time_list = stock_data_time.columns.tolist()
    for stock_code in stock_codes:
        for stock_time in stock_time_list:
            stock_open = stock_data['open'].loc[stock_code, stock_time].round(2)
            stock_high = stock_data['high'].loc[stock_code, stock_time].round(2)
            stock_low = stock_data['low'].loc[stock_code, stock_time].round(2)
            stock_close = stock_data['close'].loc[stock_code, stock_time].round(2)
            stock_volume = stock_data['volume'].loc[stock_code, stock_time]
            stock_amount = stock_data['amount'].loc[stock_code, stock_time]
            stock_obj = model_cls(
                code=stock_code,
                time=stock_time,
                open=stock_open,
                high=stock_high,
                low=stock_low,
                close=stock_close,
                volume=stock_volume,
                amount=stock_amount
            )
            stock_list.append(stock_obj)
    print(f'解析股票数据成功，解析到{len(stock_list)}条数据')

    return stock_list

In [ ]:
# 获取可用周期列表
xtdata.get_period_list()

In [ ]:
# 获取节假日数据
xtdata.download_holiday_data()

In [ ]:
holidays: List = xtdata.get_holidays()
print(f'节假日数据：{holidays}')

In [ ]:
# 获取交易日历
xtdata.get_trading_calendar('SH', start_time='20250101', end_time='20251231')

In [ ]:
# 获取交易时段
xtdata.get_all_trading_periods()

In [ ]:
# 下载股票的日K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的日 K 数据成功")
# 下载股票的周K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1w',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的周 K 数据成功")
# 下载股票的月K 行情数据
xtdata.download_history_data(
    stock_code=stock_code,
    period='1mon',
    start_time=one_years_ago,
    end_time=today,
    incrementally=False)
print(f"下载股票 {stock_code} 的月 K 数据成功")

In [ ]:
# 测试获取下载的行情数据--获取最近10条数据
stock_code: str = '000001.SZ'
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')
# 解析返回数据
"""
{field1 : value1, field2 : value2, ...}
            field1, field2, ... : 数据字段
            value1, value2, ... : pd.DataFrame 字段对应的数据，各字段维度相同，index为stock_list，columns为time_list
"""
# 获取time字段的数据
stock_data_time: pd.DataFrame = stock_data['time']

stock_data_time.head()

In [ ]:
# 获取下载的行情数据--获取所有数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1w',
    start_time=one_years_ago,
    end_time=today,
    count=-1,
    # dividend_type='none',  # 不获取分红派息数据
    dividend_type='front',  # 获取前复权数据
    fill_data=False
)
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据{len(stock_data)}格式：{type(stock_data)}')
stock_data['high'].head()  # 查看返回数据的格式

In [ ]:
# 测试未下载的股票行情数据能否获取
stock_code = '000001.SZ'  # 测试未下载的股票代码
print(f'开始获取股票 {stock_code} 的行情数据')
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=three_years_ago,
    end_time=two_years_ago,
    count=-1,
    dividend_type='none',
    fill_data=False
)
stock_data['time'].head()  # 查看返回数据的格式

In [ ]:
# 测试能否够获取前复权的股票行情数据
print(f"开始获取股票 {stock_code} 的前复权行情数据")
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=10,  # 获取最近10条数据
    dividend_type='front',  # 获取前复权数据 0530:11.108 0603:11.178 0612:11.56 0613：11.68
    # dividend_type='none',  # 获取无复权数据 0530:11.47 0603:11.54 0612:11.56 0613:11.68
    fill_data=False
)
stock_data['open'].head()

In [ ]:
# 计算一下“沪深A股”成分股最近三年的日K行情数量量
a_sector_name = '沪深A股'
stock_list: List = xtdata.get_stock_list_in_sector(a_sector_name)
print(f'获取板块 {a_sector_name} 成分股列表成功，共有 {len(stock_list)} 支股票')

In [ ]:
# 遍历成分股列表，下载每支股票的日K行情数据
for stock_code in stock_list:
    print(f'开始下载股票 {stock_code} 的日K行情数据')
    xtdata.download_history_data(
        stock_code=stock_code,
        period='1d',
        start_time=three_years_ago,
        end_time=today,
        incrementally=False
    )
    print(f'下载板块 {a_sector_name} 成分股{stock_code}的日K行情数据成功')

In [ ]:
a300_sector_name = '沪深300'
a300_stock_list: List = xtdata.get_stock_list_in_sector(a300_sector_name)
print(f'获取板块 {a300_sector_name} 成分股列表成功，共有 {len(a300_stock_list)} 支股票')

In [ ]:
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount'],
    stock_list=['002415.SZ'],
    period='1d',
    start_time=three_years_ago,
    end_time=today,
    count=-1,
    dividend_type='none',
    fill_data=False
)
print(f'获取股票 002415.SZ 的日K行情数据成功，共有 {len(stock_data["time"].columns.tolist())} 条数据')

In [ ]:
# 计算沪深300成分股最近三年的日K行情数量-已经下载好了
d_300_count: int = 0
for a300_sector_code in a300_stock_list:
    stock_data = xtdata.get_market_data(
        field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
        stock_list=[a300_sector_code],
        period='1d',
        start_time=three_years_ago,
        end_time=today,
        count=-1,
        dividend_type='front',
        fill_data=False
    )
    # 计算该股票的日K行情数量
    stock_objs = parse_stock_data(stock_data, QmtStockDailyOri)

    d_300_count += len(stock_objs)

print(f'沪深300成分股最近三年的日K行情数量为：{d_300_count}')

In [ ]:
stock_data = xtdata.get_market_data(
        field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
        stock_list=a300_stock_list,
        period='1d',
        start_time=three_years_ago,
        end_time=today,
        count=-1,
        dividend_type='none',
        fill_data=False
    )
print(f'获取沪深300成分股最近三年的日K行情数据成功')
stock_objs = parse_stock_data(stock_data, QmtStockDailyOri)
print(f'解析沪深300成分股最近三年的日K行情数据成功，共有 {len(stock_objs)} 条数据')

In [30]:
# 数据下载回调函数
def download_kline_callback(data):
    print(f'下载K线数据回调: {data}')

In [31]:
def batch_download_all_stocks(stock_codes: List[str], start_time_str: str, end_time_str: str):
    """
    批量下载所有股票的历史数据

    Args:
        stock_codes: 股票代码列表
        start_time_str: 开始时间字符串
        end_time_str: 结束时间字符串
    """
    try:
        # 打印函数入参
        print(f'批量下载股票代码列表: {len(stock_codes)},开始时间: {start_time_str}, 结束时间: {end_time_str}')
        print(f"开始批量下载{len(stock_codes)}只股票的历史数据，时间范围：{start_time_str} - {end_time_str}")

        # 批量下载所有股票数据
        xtdata.download_history_data2(
            stock_list=stock_codes,
            period='1d',
            start_time=start_time_str,
            end_time=end_time_str,
            callback=download_kline_callback,
            incrementally=False  # 使用增量下载
        )

        print(f"批量下载{len(stock_codes)}只股票的历史数据完成")

    except Exception as e:
        print(f"批量下载股票数据失败: {e}")
        raise

In [33]:
a300_sector_name = '沪深300'
a300_stock_list: List = xtdata.get_stock_list_in_sector(a300_sector_name)
print(f'获取板块 {a300_sector_name} 成分股列表成功，共有 {len(a300_stock_list)} 支股票')

# 提取所有股票代码
all_stock_codes = [sector_stock
                   for sector_stock in a300_stock_list]
total_stocks = len(all_stock_codes)
print(f"获取到沪深A股成分股共{total_stocks}只股票")
# 取一周前的日期
begin_time = (datetime.now() - timedelta(days=20))
print(f'begin_time为：{begin_time}')

# 获取今天日期
today = datetime.now()
print(f'今天的日期为：{today}')
# 1. 前置批量下载所有股票的历史数据
start_time_str = begin_time.strftime('%Y%m%d')
end_time_str = today.strftime('%Y%m%d')

***** xtdata连接成功 2025-06-24 16:23:36*****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: C:\Users\19167\AppData\Local\Programs\gjzqqmt\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

获取板块 沪深300 成分股列表成功，共有 300 支股票
获取到沪深A股成分股共300只股票
begin_time为：2025-06-04 16:36:23.837994
今天的日期为：2025-06-24 16:36:23.838538


In [35]:
download_start_time = datetime.now()
batch_download_all_stocks(all_stock_codes, start_time_str, end_time_str)
download_end_time = datetime.now()
print(f"批量下载耗时：{download_end_time - download_start_time}")

批量下载股票代码列表: 300,开始时间: 20250604, 结束时间: 20250624
开始批量下载300只股票的历史数据，时间范围：20250604 - 20250624
下载K线数据回调: {'finished': 1, 'total': 300, 'stockcode': '', 'message': '600000.SH'}
下载K线数据回调: {'finished': 2, 'total': 300, 'stockcode': '', 'message': '600009.SH'}
下载K线数据回调: {'finished': 3, 'total': 300, 'stockcode': '', 'message': '600010.SH'}
下载K线数据回调: {'finished': 4, 'total': 300, 'stockcode': '', 'message': '600011.SH'}
下载K线数据回调: {'finished': 5, 'total': 300, 'stockcode': '', 'message': '600015.SH'}
下载K线数据回调: {'finished': 6, 'total': 300, 'stockcode': '', 'message': '600016.SH'}
下载K线数据回调: {'finished': 7, 'total': 300, 'stockcode': '', 'message': '600018.SH'}
下载K线数据回调: {'finished': 8, 'total': 300, 'stockcode': '', 'message': '600019.SH'}
下载K线数据回调: {'finished': 9, 'total': 300, 'stockcode': '', 'message': '600023.SH'}
下载K线数据回调: {'finished': 10, 'total': 300, 'stockcode': '', 'message': '600025.SH'}
下载K线数据回调: {'finished': 11, 'total': 300, 'stockcode': '', 'message': '600026.SH'}
下载K线数据回调: {'finis